In [12]:
!pip install klepto

In [13]:
import os

# file_path = '/path/to/your/file.txt'
file_path = '/kaggle/input/apoldata/data'


if os.path.exists(file_path):
    print("File exists!")
else:
    print("File does not exist.")


File exists!


In [14]:
#prediction/util.py
# -*- coding: utf-8 -*-
from __future__ import print_function, division
from torch.utils.data import Dataset, DataLoader
import scipy.io as scp
import numpy as np
import torch
import pickle
import os
from klepto.archives import dir_archive

#___________________________________________________________________________________________________________________________

def lstToCuda(lst):
    for item in lst:
        item.cuda()
    return lst

def klepto_load(loc):
    '''
    for loading the dumped dictionarys
    :return: loaded dictionary
    '''

    dic = dir_archive(loc, {}, serialized=True)
    dic.load()
    print('dictionary loaded')
    return dic

### Dataset class for the NGSIM dataset
class ngsimDataset(Dataset):

    '''
    t_h = input size (history)
    t_f = output size (future)
    '''
    def __init__(self, file_name, data_dir, track_dir, dtype, dsId, class_type, device='cuda:0', set=30, t_h=24, t_f=36, d_s=1 , enc_size = 64, grid_size = (13,3), upp_grid_size = (7,3)):
        

        self.dtype = dtype#{train, val, test}
        self.data_dir = data_dir
        self.dset = dsId#it means trajectory file
        
        print('load {} dataset'.format(self.dtype))
        print("dtype is {} and dset(manual_seed) is {}".format(self.dtype, self.dset))
        if class_type == 'vehicle':
            d= np.load(os.path.join(self.data_dir, self.dtype, "{}Set{}-traj-v.npy".format(self.dtype, self.dset)), allow_pickle=True)
        elif class_type == 'bike/motor':
            d= np.load(os.path.join(self.data_dir, self.dtype, "{}Set{}-traj-b.npy".format(self.dtype, self.dset)), allow_pickle=True)
        elif class_type == 'human':
            d= np.load(os.path.join(self.data_dir, self.dtype, "{}Set{}-traj-h.npy".format(self.dtype, self.dset)), allow_pickle=True)
        else:
            d= np.load(os.path.join(self.data_dir, self.dtype, "{}Set{}-traj.npy".format(self.dtype, self.dset)), allow_pickle=True)
        self.D = d[0]
        print("The size of TRAJ file: ", len(self.D))

        t = np.load(os.path.join(self.data_dir, self.dtype, "{}Set{}-track.npy".format(self.dtype, self.dset)), allow_pickle=True)
        self.T = t[0]
        print("The size of TRACK file: ", len(self.T))

        self.t_h = t_h  # length of track history#Ben: Input size (History)
        self.t_f = t_f  # length of predicted trajectory#Ben: output size (Future)
        self.d_s = d_s  # down sampling rate of all sequences# TODO#내가 만약 history를 6개를 보는데 d_s가 2라면 결국 띄엄띄엄해서 3개의 history를 보는 것.
        self.enc_size = enc_size # size of encoder LSTM
        self.grid_size = grid_size # size of social context grid
        self.upp_grid_size = upp_grid_size #                                           #Behavioral Modification 2: Adding Kinetic Flow layer
        self.inds = [14,15,16,17,18,19,20, 27,28,29,30,31,32,33, 40,41,42,43,44,45,46]# HORIZON; front Info.
        #-->left top + center top + right top//BEN
        # self.inds = [32,33,34,35]
        self.dwn_inds = [8,9,10,11,12,13, 21,22,23,24,25,26, 34,35,36,37,38,39]# NEIGHBOR?; rear Info.
        #-->left bottom + center bottom + right bottom//BEN
        # self.dwn_inds = [35,36,37,38, 39]
        self.device = device
        self.ddd = [128, 128, 128, 129, 133, 133, 128]#dataset ids
        self.vvv = [8, 11, 53, 21, 13, 22, 52]
        self.fff = [2, 3, 2, 48, 2, 2, 16]
        


    def __len__(self):
        return len(self.D)



    def __getitem__(self, idx):

        dsId = self.D[idx, 0].astype(int)# dsId.txt
        vehId = self.D[idx, 1].astype(int)# unique Vehicle ID in dataset
        t = self.D[idx, 2] # Frame
        current = np.array([self.D[idx, 3:5]])# x, y
        
        grid = self.D[idx,8:47] # Ben: surrounding obstacles using distance# Neighbor! TODO
        upp_grid = self.D[idx,self.inds]# Inner-distance of forward objects# Horizon! TODO
        neighbors = []
        upper_neighbors = []

        hist = self.getHistory(vehId,t,vehId,dsId,current)#현재(t) 자신(vehId)의 x,y를 기준으로 과거(t부터t_h까지)의 상대적인 dx, dy
        fut = self.getFuture(vehId,t,dsId)#현재(t) 자신(vehId)의 x,y를 기준으로 미래(t부터t_f까지)의 상대적인 dx, dy

        # Get track histories of all neighbours 'neighbors' = [ndarray,[],ndarray,ndarray]
        for i in grid:#NEIGHBOR's idx
            #현재(t) 자신(vehId)의 x,y를 기준으로 Neighbor들의 과거(t부터t_h까지)의 상대적인 dx, dy
            neighbors.append(self.getHistory(i.astype(int), t, vehId, dsId, current))            
                                                                                        #Behavioral Modification 2: Adding Kinetic Flow layer
            #현재(t) 자신(vehId)의 x,y를 기준으로 Horizon들의 과거(t부터t_h까지)의 상대적인 dx, dy
        for i in upp_grid:#HORIZON's idx
            upper_neighbors.append(self.getHistory(i.astype(int), t, vehId, dsId, current))

        upp_count = np.count_nonzero(upp_grid)# front objects의 개수
        dwn_count = np.count_nonzero(self.D[idx, self.dwn_inds])#rear objects의 개수
        hist = np.concatenate((hist, np.array([[upp_count, dwn_count]])), axis=0)#len(hist) == t_h+3+1(?)
        # Ben: hist는 결국 [자신의 t부터 t_h까지의 x,y ,
        #                  t_h-t의 1/3 지점의 x,y   ,
        #                  t_h-t의 2/3 지점의 x,y   ,
        #                  t_h-t의 3/3 지점의 x,y   ,
        #                  front, rear object의 개수]

        # Maneuvers 'lon_enc' = one-hot vector, 'lat_enc = one-hot vector
        lon_enc = np.zeros([2])
        lon_enc[int(self.D[idx, 7] - 1)] = 1#lon_enc = [0, 1]
        lat_enc = np.zeros([3])
        lat_enc[int(self.D[idx, 6] - 1)] = 1#lat_enc = [0, 0, 1]

        if dsId in self.ddd:# Don't know what it means...; maybe useless code?
            idx = 0
            while self.ddd[idx] != dsId:
                idx += 1
            if vehId == self.vvv[idx] and t == self.fff[idx]:
                bb = True
            else:
                bb = False
        else:
            bb = False

        return hist, fut, upper_neighbors, neighbors, lat_enc, lon_enc, bb, dsId, vehId, t
        '''
        hist = [자신의 t부터 t_h까지의 x,y , t_h-t의 1/3 지점의 x,y, t_h-t의 2/3 지점의 x,y ,
                    t_h-t의 3/3 지점의 x,y, front, rear object의 개수]
        fut = 현재(t) 자신(vehId)의 x,y를 기준으로 미래(t부터t_f까지)의 상대적인 dx, dy
        upp_neighbors: 현재(t) 자신(vehId)의 x,y를 기준으로 Horizon들의 과거(t부터t_h까지)의 상대적인 dx, dy
        neighbors: 현재(t) 자신(vehId)의 x,y를 기준으로 Neighbor들의 과거(t부터t_h까지)의 상대적인 dx, dy
        lon_enc: [0, 1]
        lat_enc: [0, 0, 1]
        bb: False?
        vehId: 현재 자신(vehId); object가 되겠죠.
        t: 현재 frame
        '''

    ## Helper function to get track history
    # refVeh: object's
    # Veh: the master of the current row
    def getHistory(self, vehId, t, refVehId, dsId, current):
        # 결국, refVeh의 t에서의 x,y를 기준으로, veh의 t부터 t_h까지의 상대적인 x,y의 history
        if vehId == 0:
            return current
        else:
            if not vehId in self.T[dsId].keys():
                out0 = np.full((self.t_h+3, 2), 0.0)#size==(9,2)
                return out0

            refTrack = (self.T[dsId][refVehId].transpose()).astype(float)
            vehTrack = (self.T[dsId][vehId].transpose()).astype(float)
            
            # vehId의 refVehId가 같은 frame에 있었을 때의 x,y를 수집
            refPos = refTrack[np.where(refTrack[:,0]==t)][0,1:3]#Ben: Other objects' pose at time t (at that frame)

            if vehTrack.size==0 or np.argwhere(vehTrack[:, 0] == t).size==0:
                out = np.full((self.t_h+3, 2), current+1e-3-refPos)#size==(9,2)
                return out
            else:
                #현재t보다 t_h뒤의 index. 만약, 현재 t기준으로 t_h 뒤가 trajectory 초기의 t값 wmr, t-t_h <0이면, 그때는 0의 값을 갖는다.
                stpt = np.maximum(0, np.argwhere(vehTrack[:, 0] == t).item(0)+1 - self.t_h)
                #현재t보다 하나 더 앞의 index
                enpt = np.argwhere(vehTrack[:, 0] == t).item(0) + 1
                hist = vehTrack[stpt:enpt:self.d_s,1:3]-refPos#Get only relative positions [m]
                #vehTrack에서 현재 stpt부터 enpt까지의 x,y
            if len(hist) < self.t_h//self.d_s:
                out2 = np.full((self.t_h, 2), 1e-3)
                temp_vel = np.full((3,2), 1e-3)
                out2[out2.shape[0]-hist.shape[0]:,:] = hist
                out2 = np.concatenate((out2, temp_vel), axis=0)# velocity?
                return out2
                                                                                        # Behavioral Modification 3: Change inputs
            m1 = int(self.t_h/3)#Ben: length of history / 3; 1/3지점
            m2 = 2 * m1#2/3지점,,,,, t_h는 3/3지점.
            vel0 = np.array([[hist[m1][0] - hist[0][0], hist[m1][1] -hist[0][1]]])#현재를 기준으로 우리가 보는 history길이의 1/3 전의 x,y
            vel5 = np.array([[hist[m2][0] - hist[m1][0], hist[m2][1] -hist[m1][1]]])#현재를 기준으로 우리가 보는 history길이의 2/3 전의 x,y
            vel10 = np.array([[hist[self.t_h-1][0] - hist[m2][0], hist[self.t_h-1][1] -hist[m2][1]]])#현재를 기준으로 우리가 보는 history길이의 끝에서의 x,y
            hist = np.concatenate((hist, np.concatenate((vel0,vel5,vel10), axis=0)), axis=0)
            return hist



    ## Helper function to get track future
    def getFuture(self, vehId, t,dsId):
        # vehTrack = (self.T[dsId][vehId-1].transpose()).astype(float)
        vehTrack = (self.T[dsId][vehId].transpose()).astype(float)
        refPos = vehTrack[np.where(vehTrack[:, 0] == t)][0, 1:3]
        stpt = np.argwhere(vehTrack[:, 0] == t).item(0) + self.d_s
        enpt = np.minimum(len(vehTrack), np.argwhere(vehTrack[:, 0] == t).item(0) + self.t_f + 1)
        fut = vehTrack[stpt:enpt:self.d_s,1:3]-refPos#Ben: Based on the current pose, get future gt trajectory
        # print(dsId, vehId, vehTrack[stpt:enpt:self.d_s,1:3])
        return fut#len(fut) is maybe different

    ## Collate function for dataloader
    def collate_fn(self, samples):
        
        # Initialize neighbors and neighbors length batches:
        nbr_batch_size = 0

        # upp_nbrs: horizon agent's relative poses
        # nbrs: neighbor agent's relative poses
        for _, _, upp_nbrs, nbrs, _, _, _, _, _, _ in samples:
            nbr_batch_size += sum([len(nbrs[i])!=0 for i in range(len(nbrs))])
        maxlen = self.t_h//self.d_s + 3 + 1
        nbrs_batch = torch.zeros(maxlen,nbr_batch_size, 2)
                                                                                                    # Behavioral Modification 2: Adding Kinetic Flow layer
        upp_nbr_batch_size = 0
        for _, _, upp_nbrs, nbrs,_ ,_ ,_ ,_ ,_ ,_ in samples:
            upp_nbr_batch_size += sum([len(upp_nbrs[i])!=0 for i in range(len(upp_nbrs))])
        upp_maxlen = self.t_h//self.d_s + 3 + 1                                                               # Behavioral Modification 3: Change inputs/ change max len to +3
        upp_nbrs_batch = torch.zeros(upp_maxlen,upp_nbr_batch_size,2)

        # Initialize social mask batch:#grid_size = (13,3), upp_grid_size = (7,3)
        pos = [0, 0]
        mask_batch = torch.zeros(len(samples), self.grid_size[1], self.grid_size[0], self.enc_size)
        mask_batch = mask_batch.byte()

        upp_pos = [0,0]                                                                                 # Behavioral Modification 2: Adding Kinetic Flow layer
        upp_mask_batch = torch.zeros(len(samples), self.upp_grid_size[1], self.upp_grid_size[0], self.enc_size)
        upp_mask_batch = upp_mask_batch.byte()


        hist_batch = torch.zeros(maxlen,len(samples),2)
        fut_batch = torch.zeros(self.t_f//self.d_s,len(samples),2)
        fut_mask_batch = torch.zeros(self.t_f//self.d_s,len(samples),2)
        lat_enc_batch = torch.zeros(len(samples),3)
        lon_enc_batch = torch.zeros(len(samples), 2)
        bb_batch = np.zeros(len(samples))
        dd_batch = np.zeros(len(samples))
        vv_batch = np.zeros(len(samples))
        ff_batch = np.zeros(len(samples))

        count = 0
        upp_count = 0
        # each sample: hist,fut,upper_neighbors, neighbors,lat_enc,lon_enc, bb, dsId, vehId, t
        for sampleId,(hist, fut, upp_nbrs, nbrs, lat_enc, lon_enc, bb, dd, vv, ff) in enumerate(samples):
            # Set up history, future, lateral maneuver and longitudinal maneuver batches:
            hist_batch[0:len(hist),sampleId,0] = torch.from_numpy(hist[:, 0])   #x
            hist_batch[0:len(hist), sampleId, 1] = torch.from_numpy(hist[:, 1]) #y
            fut_batch[0:len(fut), sampleId, 0] = torch.from_numpy(fut[:, 0])
            fut_batch[0:len(fut), sampleId, 1] = torch.from_numpy(fut[:, 1])
            fut_mask_batch[0:len(fut),sampleId,:] = 1# if enough future doesn't exist --> zero
            lat_enc_batch[sampleId,:] = torch.from_numpy(lat_enc)
            lon_enc_batch[sampleId, :] = torch.from_numpy(lon_enc)
            bb_batch[sampleId] = bb
            dd_batch[sampleId] = dd
            vv_batch[sampleId] = vv
            ff_batch[sampleId] = ff

            # Set up neighbor, neighbor sequence length, and mask batches:
            for id,nbr in enumerate(nbrs):
                if len(nbr)!=0:#if there is a history of nbrs
                    nbrs_batch[0:len(nbr), count,0] = torch.from_numpy(nbr[:, 0])#x
                    nbrs_batch[0:len(nbr), count, 1] = torch.from_numpy(nbr[:, 1])#y
                    pos[0] = id % self.grid_size[0]#13
                    pos[1] = id // self.grid_size[0]#13
                    mask_batch[sampleId,pos[1],pos[0],:] = torch.ones(self.enc_size).byte()
                    count+=1#each count, each neighbor

                                                                                                   # Behavioral Modification 2: Adding Kinetic Flow layer
            for id, upp_nbr in enumerate(upp_nbrs):
                if len(upp_nbr) != 0:
                    upp_nbrs_batch[0:len(upp_nbr), upp_count, 0] = torch.from_numpy(upp_nbr[:, 0])
                    upp_nbrs_batch[0:len(upp_nbr), upp_count, 1] = torch.from_numpy(upp_nbr[:, 1])
                    upp_pos[0] = id % self.upp_grid_size[0]#7
                    upp_pos[1] = id // self.upp_grid_size[0]
                    upp_mask_batch[sampleId, upp_pos[1], upp_pos[0], :] = torch.ones(self.enc_size).byte()#byte() just declares
                    upp_count += 1

        # print('it is from the collate_fn', hist_batch)
        return hist_batch, upp_nbrs_batch, nbrs_batch, upp_mask_batch, mask_batch, lat_enc_batch, lon_enc_batch, fut_batch, fut_mask_batch, bb_batch, dd_batch, vv_batch, ff_batch

#________________________________________________________________________________________________________________________________________





## Custom activation for output layer (Graves, 2015)
def outputActivation(x):
    muX = x[:,:,0:1]
    muY = x[:,:,1:2]
    sigX = x[:,:,2:3]
    sigY = x[:,:,3:4]
    rho = x[:,:,4:5]
    sigX = torch.exp(sigX)
    sigY = torch.exp(sigY)
    rho = torch.tanh(rho)
    out = torch.cat([muX, muY, sigX, sigY, rho],dim=2)
    return out

## Batchwise NLL loss, uses mask for variable output lengths
def maskedNLL(y_pred, y_gt, mask, device='cpu'):
    acc = torch.zeros_like(mask, device=device)
#     print(f"y_pred value: {y_pred}")
    muX = y_pred[:,:,0]
    muY = y_pred[:,:,1]
    sigX = y_pred[:,:,2]
    sigY = y_pred[:,:,3]
    rho = y_pred[:,:,4]
    ohr = torch.pow(1-torch.pow(rho,2),-0.5)
    x = y_gt[:,:, 0]
    y = y_gt[:,:, 1]
    out = torch.pow(ohr, 2)*(torch.pow(sigX, 2)*torch.pow(x-muX, 2) + torch.pow(sigY, 2)*torch.pow(y-muY, 2) - 2*rho*torch.pow(sigX, 1)*torch.pow(sigY, 1)*(x-muX)*(y-muY)) - torch.log(sigX*sigY*ohr)
    acc[:,:,0] = out
    acc[:,:,1] = out
    acc = acc*mask
#    print(acc[0])
    lossVal = torch.sum(acc)/torch.sum(mask)
    return lossVal

## NLL for sequence, outputs sequence of NLL values for each time-step, uses mask for variable output lengths, used for evaluation
def maskedNLLTest(fut_pred, lat_pred, lon_pred, fut, fut_mask, device='cpu',num_lat_classes=3, num_lon_classes = 2,use_maneuvers = True, avg_along_time = False, cuda=True):
    if use_maneuvers:
        if cuda:
            acc = torch.zeros(fut_mask.shape[0],fut_mask.shape[1],num_lon_classes*num_lat_classes).cuda(device)
        else:
            acc = torch.zeros(fut_mask.shape[0],fut_mask.shape[1],num_lon_classes*num_lat_classes)
        count = 0
        for k in range(num_lon_classes):
            for l in range(num_lat_classes):
                wts = lat_pred[:,l]*lon_pred[:,k]
                wts = wts.repeat(len(fut_pred[0]),1)
                y_pred = fut_pred[k*num_lat_classes + l]
                y_gt = fut
                muX = y_pred[:, :, 0]
                muY = y_pred[:, :, 1]
                sigX = y_pred[:, :, 2]
                sigY = y_pred[:, :, 3]
                rho = y_pred[:, :, 4]
                ohr = torch.pow(1 - torch.pow(rho, 2), -0.5)
                x = y_gt[:, :, 0]
                y = y_gt[:, :, 1]
                out = -(torch.pow(ohr, 2) * (torch.pow(sigX, 2) * torch.pow(x - muX, 2) + torch.pow(sigY, 2) * torch.pow(y - muY,2) - 2 * rho * torch.pow(sigX, 1) * torch.pow(sigY, 1) * (x - muX) * (y - muY)) - torch.log(sigX * sigY * ohr))
                acc[:, :, count] =  out + torch.log(wts)
                count+=1
        acc = -logsumexp(acc,dim = 2)
        acc = acc * fut_mask[:,:,0]
        if avg_along_time:
            lossVal = torch.sum(acc) / torch.sum(fut_mask[:, :, 0])
            return lossVal
        else:
            lossVal = torch.sum(acc,dim=1)
            counts = torch.sum(fut_mask[:,:,0],dim=1)
            return lossVal,counts
    else:
        if cuda:
            acc = torch.zeros(fut_mask.shape[0], fut_mask.shape[1], 1).cuda(device)
        else:
            acc = torch.zeros(fut_mask.shape[0], fut_mask.shape[1], 1)
        y_pred = fut_pred
        y_gt = fut
        muX = y_pred[:, :, 0]
        muY = y_pred[:, :, 1]
        sigX = y_pred[:, :, 2]
        sigY = y_pred[:, :, 3]
        rho = y_pred[:, :, 4]
        ohr = torch.pow(1 - torch.pow(rho, 2), -0.5)
        x = y_gt[:, :, 0]
        y = y_gt[:, :, 1]
        out = torch.pow(ohr, 2) * (
        torch.pow(sigX, 2) * torch.pow(x - muX, 2) + torch.pow(sigY, 2) * torch.pow(y - muY, 2) - 2 * rho * torch.pow(
            sigX, 1) * torch.pow(sigY, 1) * (x - muX) * (y - muY)) - torch.log(sigX * sigY * ohr)
        acc[:, :, 0] = out
        acc = acc * fut_mask[:, :, 0:1]
        if avg_along_time:
            lossVal = torch.sum(acc[:, :, 0]) / torch.sum(fut_mask[:, :, 0])
            return lossVal
        else:
            lossVal = torch.sum(acc[:,:,0], dim=1)
            counts = torch.sum(fut_mask[:, :, 0], dim=1)
            return lossVal,counts

## Batchwise MSE loss, uses mask for variable output lengths
def maskedMSE(y_pred, y_gt, mask, device='cpu'):
    acc = torch.zeros_like(mask, device=device)
#     print(f"y_pred value: {y_pred}")
    muX = y_pred[:,:,0]
    muY = y_pred[:,:,1]
    x = y_gt[:,:, 0]
    y = y_gt[:,:, 1]
    out = torch.pow(x-muX, 2) + torch.pow(y-muY, 2)
    acc[:,:,0] = out
    acc[:,:,1] = out
    acc = acc*mask
#    print(acc)
    lossVal = torch.sum(acc)/torch.sum(mask)
    return lossVal

## MSE loss for complete sequence, outputs a sequence of MSE values, uses mask for variable output lengths, used for evaluation
def maskedMSETest(y_pred, y_gt, mask, device='cpu'):
    acc = torch.zeros_like(mask, device=device)
    muX = y_pred[:, :, 0]
    muY = y_pred[:, :, 1]
    x = y_gt[:, :, 0]
    y = y_gt[:, :, 1]
    out = torch.pow(x - muX, 2) + torch.pow(y - muY, 2)
    acc[:, :, 0] = out
    acc[:, :, 1] = out
    acc = acc * mask
    lossVal = torch.sum(acc[:,:,0],dim=1)
    counts = torch.sum(mask[:,:,0],dim=1)
    return lossVal, counts

## Helper function for log sum exp calculation:
def logsumexp(inputs, dim=None, keepdim=False):
    if dim is None:
        inputs = inputs.view(-1)
        dim = 0
    s, _ = torch.max(inputs, dim=dim, keepdim=True)
    outputs = s + (inputs - s).exp().sum(dim=dim, keepdim=True).log()
    if not keepdim:
        outputs = outputs.squeeze(dim)
    return outputs

print("Execute done")

Execute done


In [15]:
#prediction/social.py
from __future__ import division
import torch
from torch.autograd import Variable
import torch.nn as nn
# from model.Prediction.utils import outputActivation

class highwayNet(nn.Module):

    ## Initialization
    def __init__(self,args):
        super(highwayNet, self).__init__()

        ## Unpack arguments
        self.args = args

        ## Use gpu flag
        self.use_cuda = args['cuda']

        # Flag for maneuver based (True) vs uni-modal decoder (False)
        self.use_maneuvers = args['use_maneuvers']

        # Flag for train mode (True) vs test-mode (False)
        self.train_flag = True

        ## Sizes of network layers
        self.encoder_size = args['encoder_size']
        self.decoder_size = args['decoder_size']
        self.in_length = args['in_length']
        self.out_length = args['out_length']
        self.grid_size = args['grid_size']
        self.soc_conv_depth = args['soc_conv_depth']
        self.conv_3x1_depth = args['conv_3x1_depth']
        self.dyn_embedding_size = args['dyn_embedding_size']
        self.input_embedding_size = args['input_embedding_size']
        self.num_lat_classes = args['num_lat_classes']
        self.num_lon_classes = args['num_lon_classes']
        self.soc_embedding_size = (((args['grid_size'][0]-4)+1)//2)*self.conv_3x1_depth

        ## Define network weights

        # Input embedding layer
        self.ip_emb = torch.nn.Linear(2,self.input_embedding_size)

        # Encoder LSTM
        self.enc_lstm = torch.nn.LSTM(self.input_embedding_size,self.encoder_size,1)
#         self.enc_gru = torch.nn.GRU(self.input_embedding_size,self.encoder_size,1)
        
        # Vehicle dynamics embedding
        self.dyn_emb = torch.nn.Linear(self.encoder_size,self.dyn_embedding_size)

        # Convolutional social pooling layer and social embedding layer
        self.soc_conv = torch.nn.Conv2d(self.encoder_size,self.soc_conv_depth,3)
        self.conv_3x1 = torch.nn.Conv2d(self.soc_conv_depth, self.conv_3x1_depth, (3,1))
        self.soc_maxpool = torch.nn.MaxPool2d((2,1),padding = (1,0))

        # FC social pooling layer (for comparison):
        # self.soc_fc = torch.nn.Linear(self.soc_conv_depth * self.grid_size[0] * self.grid_size[1], (((args['grid_size'][0]-4)+1)//2)*self.conv_3x1_depth)

        # Decoder LSTM
        if self.use_maneuvers:
            self.dec_lstm = torch.nn.GRU(self.soc_embedding_size + self.dyn_embedding_size + self.num_lat_classes + self.num_lon_classes, self.decoder_size)
        else:
            self.dec_lstm = torch.nn.GRU(self.soc_embedding_size + self.dyn_embedding_size, self.decoder_size)

        # Output layers:
        self.op = torch.nn.Linear(self.decoder_size,5)
        self.op_lat = torch.nn.Linear(self.soc_embedding_size + self.dyn_embedding_size, self.num_lat_classes)
        self.op_lon = torch.nn.Linear(self.soc_embedding_size + self.dyn_embedding_size, self.num_lon_classes)

        # Activations:
        self.leaky_relu = torch.nn.LeakyReLU(0.1)
        self.relu = torch.nn.ReLU()
        self.softmax = torch.nn.Softmax(dim=1)

#        self.bn_dec = torch.nn.BatchNorm1d(self.soc_embedding_size + self.dyn_embedding_size) 


    ## Forward Pass
    def forward(self,hist,nbrs,masks,lat_enc,lon_enc):
        
        ## Forward pass hist:
        _,(hist_enc,_) = self.enc_lstm(self.leaky_relu(self.ip_emb(hist)))
        hist_enc = self.leaky_relu(self.dyn_emb(hist_enc.view(hist_enc.shape[1],hist_enc.shape[2])))
        ## Forward pass nbrs
        
        _, (nbrs_enc,_) = self.enc_lstm(self.leaky_relu(self.ip_emb(nbrs)))
        nbrs_enc = nbrs_enc.view(nbrs_enc.shape[1], nbrs_enc.shape[2])
        ## Masked scatter
        masks2 = masks.bool()
        soc_enc = torch.zeros_like(masks).float()
        soc_enc = soc_enc.masked_scatter_(masks2, nbrs_enc)
        soc_enc = soc_enc.permute(0,3,2,1)

        ## Apply convolutional social pooling:
        soc_enc = self.soc_maxpool(self.leaky_relu(self.conv_3x1(self.leaky_relu(self.soc_conv(soc_enc)))))
        soc_enc = soc_enc.view(-1,self.soc_embedding_size)

        ## Apply fc soc pooling
        # soc_enc = soc_enc.contiguous()
        # soc_enc = soc_enc.view(-1, self.soc_conv_depth * self.grid_size[0] * self.grid_size[1])
        # soc_enc = self.leaky_relu(self.soc_fc(soc_enc))
        ## Concatenate encodings:
        enc = torch.cat((soc_enc,hist_enc),1)


        if self.use_maneuvers:
            ## Maneuver recognition:
            lat_pred = self.softmax(self.op_lat(enc))
            lon_pred = self.softmax(self.op_lon(enc))

            if self.train_flag:
                ## Concatenate maneuver encoding of the true maneuver
                enc = torch.cat((enc, lat_enc, lon_enc), 1)
                fut_pred = self.decode(enc)
                return fut_pred, lat_pred, lon_pred
            else:
                fut_pred = []
                ## Predict trajectory distributions for each maneuver class
                for k in range(self.num_lon_classes):
                    for l in range(self.num_lat_classes):
                        lat_enc_tmp = torch.zeros_like(lat_enc)
                        lon_enc_tmp = torch.zeros_like(lon_enc)
                        lat_enc_tmp[:, l] = 1
                        lon_enc_tmp[:, k] = 1
                        enc_tmp = torch.cat((enc, lat_enc_tmp, lon_enc_tmp), 1)
                        fut_pred.append(self.decode(enc_tmp))
                return fut_pred, lat_pred, lon_pred
        else:
#            enc = self.bn_dec(enc)
            fut_pred = self.decode(enc)
            return fut_pred


    def decode(self,enc):
        enc = enc.repeat(self.out_length, 1, 1)
        h_dec, _ = self.dec_lstm(enc)
        h_dec = h_dec.permute(1, 0, 2)
        fut_pred = self.op(h_dec)
        fut_pred = fut_pred.permute(1, 0, 2)
        fut_pred = outputActivation(fut_pred)
        return fut_pred

print("Execute done")

Execute done


In [16]:
#prediction/trajPredEngine
# -*- coding: utf-8 -*-
from ignite.engine import Engine, Events
# from model.Prediction.utils import lstToCuda,maskedNLL,maskedMSE,maskedNLLTest, maskedMSETest
import math
import torch
from ignite.contrib.handlers import ProgressBar
import os
import numpy as np
from tensorboardX import SummaryWriter
import matplotlib.pylab as plt

class TrajPredEngine:

    def __init__(self, net, optim, train_loader, val_loader, args):
        self.net = net
        self.args = args
        self.pretrainEpochs = args["pretrainEpochs"]
        self.trainEpochs = args["trainEpochs"]
        self.optim = optim
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.cuda = args['cuda']
        self.device = args['device']
        self.dsId = self.args['dsId']
        self.n_iterations = max(len(train_loader), len(train_loader) / args["batch_size"])

        ## training metrics to keep track of, consider making a metrics class
        # remember to 0 these out
        self.avg_trn_loss = 0

        self.metrics = {"Avg train loss": 0, "Avg val loss": 0 }
        ## validation metrics
        self.avg_val_loss = 0
        self.val_batch_count = 1

        # only if using maneuvers
        self.avg_lat_acc = 0
        self.avg_lon_acc = 0

        self.trainer = None
        self.evaluator = None

        self.makeTrainer()

        self.save_name = args['name']

        # testing stuff wow need 2 clean this so bad

        self.lossVals = torch.zeros(self.args['out_length']).cuda(self.device) if self.cuda else torch.zeros(self.args['out_length'])
        self.counts = torch.zeros(self.args['out_length']).cuda(self.device) if self.cuda else torch.zeros(self.args['out_length'])
        self.lastTestLoss = 0
        
        self.writer = None
        self.log_dir = args['log_dir']
        self.tensorboard = args['tensorboard']

        # Ben
        self.current_best = 1000
        self.best_epoch = 0

    def netPred(self, batch):
        raise NotImplementedError

    def saveModel(self, engine):

        os.makedirs(self.args['modelLoc'], exist_ok=True)
        name = os.path.join(self.args['modelLoc'], "epochs.{}.".format(engine.state.epoch)+self.args['name'])
        torch.save(self.net.state_dict(), name)
        print("Model saved {}.".format(name))

    def train_a_batch(self, engine, batch):
        # each sample: hist,fut,upper_neighbors, neighbors,lat_enc,lon_enc, bb, dsId, vehId, t
        self.net.train_flag = True
        epoch = engine.state.epoch

        _, _, _, _, _, _, _, fut, op_mask, _, _, _, _ = batch

        fut_pred = self.netPred(batch)
        
        if self.cuda:
            fut = fut.cuda(self.device)
            op_mask = op_mask.cuda(self.device)


        if self.args['nll_only']:
            l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
        elif epoch < self.pretrainEpochs:
            if self.args["pretrain_loss"] == 'MSE':
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
            elif self.args['pretrain_loss'] == 'NLL':
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
            else:
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
        else:
            if self.args["train_loss"] == 'MSE':
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
            elif self.args['train_loss'] == 'NLL':
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
            else:
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)

        # if self.args['nll_only']:
        #     l = maskedNLL(fut_pred, fut, op_mask)
        # else:
        #     if epoch < self.pretrainEpochs:
        #         l = maskedMSE(fut_pred, fut, op_mask)
        #     else:
        #         l = maskedNLL(fut_pred, fut, op_mask)

        # Backprop and update weights
#        if l.item() != l.item():
#            print(l.item())
#            exit(1)
#            return 1
        self.optim.zero_grad()
        l.backward()
        self.optim.step()

        # Track average train loss:
        self.avg_trn_loss += l.item()
        self.metrics["Avg train loss"] += l.item() / 100.0
           
        # if self.writer:#Ben: if you want to write down the loss, every epoch
            # print("writing...")
            # self.writer.add_scalar("{}epoch/trainingloss".format(engine.state.epoch), l.item() , engine.state.iteration)
            # def saveModel(self, engine):
            #     os.makedirs(self.args['modelLoc'], exist_ok=True)
            #     name = os.path.join(self.args['modelLoc'], self.args['name'])
            #     torch.save(self.net.state_dict(), name)
            #     print("Model saved {}.".format(name))


        return l.item()

    def eval_a_batch(self, engine, batch):
        self.net.train_flag = False

        epoch = engine.state.epoch

        _, _, _, _, _, _, _, fut, op_mask, _, _, _, _ = batch
        fut_pred = self.netPred(batch)
        if self.cuda:
            fut = fut.cuda(self.device)
            op_mask = op_mask.cuda(self.device)

        # Forward pass
        if self.args['nll_only']:
            l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
        elif epoch < self.pretrainEpochs:
            if self.args["pretrain_loss"] == 'MSE':
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
            elif self.args['pretrain_loss'] == 'NLL':
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
            else:
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
        else:
            if self.args["train_loss"] == 'MSE':
                l = maskedMSE(fut_pred, fut, op_mask, device=self.device)
            elif self.args['train_loss'] == 'NLL':
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)
            else:
                l = maskedNLL(fut_pred, fut, op_mask, device=self.device)


        # if self.args['nll_only']:
        #     l = maskedNLL(fut_pred, fut, op_mask)
        # else:
        #     if epoch_num < pretrainEpochs:
        #         l = maskedMSE(fut_pred, fut, op_mask)
        #     else:
        #         l = maskedNLL(fut_pred, fut, op_mask)

        self.avg_val_loss += l.item()
        self.metrics["Avg val loss"] += l.item()/ (self.val_batch_count * 100.0)
        self.val_batch_count += 1

        return fut_pred, fut

    def validate(self, engine):
        self.evaluator.run(self.val_loader)
        max_epochs =self.args["pretrainEpochs"] + self.args["trainEpochs"]

        # if not self.eval_only:
        print("{}/{} Epochs in dataset{}".format(engine.state.epoch, max_epochs, self.dsId))
        # print(max((engine.state.epoch / max_epochs) * 100,1))
        print("EPOCH {}: Train loss: {}  Val loss: {}\n".format(engine.state.epoch, self.metrics["Avg train loss"], self.metrics["Avg val loss"]))
        # else:
        #     print("EPOCH {}: Test loss: {}\n".format(engine.state.epoch, self.metrics["Avg val loss"]))
        if self.metrics['Avg val loss'] == min(self.metrics['Avg val loss'], self.current_best):
            self.best_epoch = engine.state.epoch
            self.current_best = self.metrics['Avg val loss']
        print("CURRENT BEST: ", self.best_epoch)
        if self.writer:
            self.writer.add_scalar("training_avg_loss", self.metrics['Avg train loss'], engine.state.epoch)
            self.writer.add_scalar("validating_avg_loss", self.metrics['Avg val loss'], engine.state.epoch)

        self.metrics["Avg train loss"] = 0
        self.metrics["Avg val loss"] = 0

    def zeroMetrics(self, engine):
        self.val_batch_count = 1
        self.metrics["Avg val loss"] = 0 

    def zeroTrainLoss(self, engine):
        self.metrics["Avg train loss"] = 0

    def zeroValLoss(self, engine):
        self.metrics["Avg val loss"] = 0

    def makeTrainer(self):
        self.trainer = Engine(self.train_a_batch)
        self.evaluator = Engine(self.eval_a_batch)

        pbar = ProgressBar(persist=True, postfix=self.metrics)
        pbar.attach(self.trainer)
        pbar.attach(self.evaluator)

        ## attach hooks 
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self.validate)
        self.trainer.add_event_handler(Events.EPOCH_COMPLETED, self.saveModel)
        self.trainer.add_event_handler(Events.ITERATION_COMPLETED, self.zeroMetrics)
        self.trainer.add_event_handler(Events.COMPLETED, self.saveModel)
        # zero out metrics for next epoch


    def create_summary_writer(self, model, data_loader, log_dir):
        writer = SummaryWriter(log_dir=log_dir)
        data_loader_iter = iter(data_loader)
        b = next(data_loader_iter)
        b = tuple(x for x in b)
#         b = tuple(x.cuda(self.device) for x in b)
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        b_gpu = tuple(x.to(device) for x in b[:7])
        try:
            writer.add_graph(model, b_gpu[:7])
        except Exception as e:
            print("Failed to save model graph: {}".format(e))
        return writer

    def start(self):
        max_epochs =self.args["pretrainEpochs"] + self.args["trainEpochs"]

        if self.tensorboard:
            self.writer = self.create_summary_writer(self.net, self.train_loader, self.log_dir)

        @self.trainer.on(Events.ITERATION_COMPLETED)
        def log_training_loss(engine):
            iter = (engine.state.iteration - 1) % len(self.train_loader) + 1
            self.writer.add_scalar("training/loss", engine.state.output, engine.state.iteration)
        #    if iter % 10 == 0:
#         print("============Running============")
        # if not self.eval_only:
        self.trainer.run(self.train_loader, max_epochs=max_epochs)
        # else:
            # self.trainer.run(self.train_loader, max_epochs=1)

        if self.tensorboard:
            self.writer.close()


    def test_a_batch(self, engine, batch):
        # print("test a batch------------==================================")
        # _, _, _, _, _, _, _, fut, op_mask, _, _, _, _ = batch
        hist_trj, _, _, _, _, _, _, fut, op_mask, _, _, vehId, _ = batch


        # Initialize Variables
        if self.cuda:
            fut = fut.cuda(self.device)
            op_mask = op_mask.cuda(self.device)

        if self.args["train_loss"] == 'NLL':
            # Forward pass
            if self.args['use_maneuvers']:
                fut_pred, lat_pred, lon_pred = self.netPred(batch)
                l,c = maskedNLLTest(fut_pred, lat_pred, lon_pred, fut, op_mask, device=self.device, cuda=self.args.cuda)
            else:
                fut_pred = self.netPred(batch)
                l, c = maskedNLLTest(fut_pred, 0, 0, fut, op_mask, device=self.device, use_maneuvers=False, cuda=self.cuda)
        else:
            # Forward pass
            if self.args['use_maneuvers']:
                fut_pred, lat_pred, lon_pred = self.netPred(batch)
                fut_pred_max = torch.zeros_like(fut_pred[0])
                for k in range(lat_pred.shape[0]):
                    lat_man = torch.argmax(lat_pred[k, :]).detach()
                    lon_man = torch.argmax(lon_pred[k, :]).detach()
                    indx = lon_man*3 + lat_man
                    fut_pred_max[:,k,:] = fut_pred[indx][:,k,:]
                l, c = maskedMSETest(fut_pred_max, fut, op_mask, device=self.device)
            else:
                fut_pred = self.netPred(batch)            
                l, c = maskedMSETest(fut_pred, fut, op_mask, device=self.device)
        fut_pred_info = fut_pred.cpu().detach().numpy()##BEN
        
        
        # print("\n vehicle Id from the net: ", vehId)
        # print("hist_traj: ", hist_trj)
        # print("GT_trj: ", fut)
        # print("Net_trj: ", fut_pred_info[:,:,:2])

        # # print("future_prediction: ", fut_pred_info.shape)
        # quit()
        self.lossVals +=l.detach()
        self.lastTestLoss = l.detach()
        self.counts += c.detach()



    def eval(self, test_loader):


        self.test_batch_size = len(test_loader)
        tester = Engine(self.test_a_batch)

        pbar = ProgressBar(persist=True, postfix=self.metrics)
        pbar.attach(tester)
        print('evaluating on dataset{}...'.format(self.dsId))
        tester.run(test_loader)
        
        if(self.args["train_loss"]) == "NLL" :
            nll_loss = self.lossVals / self.counts
            nll_loss[nll_loss != nll_loss] = 0
            print("Last Test loss: " + str(self.lastTestLoss.mean().item()))
            print("Avg Test loss: " + str(nll_loss.mean().item()))
        else:
            rmse = torch.pow(self.lossVals / self.counts, 0.5) * .3048 # converting from feet to meters
            rmse[torch.isnan(rmse)] = 0
            # self.lastTestLoss = torch.pow(self.lastTestLoss, 0.5) * .3048
            # print(self.lastTestLoss)
            seq_loss = rmse.tolist()
            seq_loss = [x for x in seq_loss if x != 0]
            print(rmse)
            print("Last Test loss: " + str(seq_loss[-1]))
            print("Avg Test loss: " + str(rmse.mean().item()))

print("Execute done")

Execute done


In [17]:
#prediction/traphicEngine
# from model.Prediction.trajPredEngine import TrajPredEngine
import torch
import datetime
import numpy as np
class TraphicEngine(TrajPredEngine):
    """
    Implementation of abstractEngine for traphic
    TODO:maneuver metrics, too much duplicate code with socialEngine
    """

    def __init__(self, net, optim, train_loader, val_loader, args):
        super().__init__(net, optim, train_loader, val_loader, args)
        self.save_name = "traphic"

    def netPred(self, batch):
        hist, upp_nbrs, nbrs, upp_mask, mask, lat_enc, lon_enc, fut, fut_mask, b, d, v, f = batch

        if self.args['cuda']:
            hist = hist.cuda(self.device)
            nbrs = nbrs.cuda(self.device)
            upp_nbrs = upp_nbrs.cuda(self.device)
            mask = mask.cuda(self.device)
            upp_mask = upp_mask.cuda(self.device)
            lat_enc = lat_enc.cuda(self.device)
            lon_enc = lon_enc.cuda(self.device)
            fut = fut.cuda(self.device)
            fut_mask = fut_mask.cuda(self.device)
        
        fut_pred  = self.net(hist, upp_nbrs, nbrs, upp_mask, mask, lat_enc, lon_enc)

        return fut_pred

print("Execute done")

Execute done


In [18]:
#prediction/socialEngine.py
# from model.Prediction.trajPredEngine import TrajPredEngine
import torch

class SocialEngine(TrajPredEngine):
    """
    Implementation of abstractEngine for traphic
    TODO:maneuver metrics, too much duplicate code with socialEngine
    """

    def __init__(self, net, optim, train_loader, val_loader, args):
        super().__init__(net, optim, train_loader, val_loader, args)
        self.save_name = "social"

    def netPred(self, batch):
        hist, _, nbrs, _, mask, lat_enc, lon_enc, _, _, b, d, v, f = batch
        if self.args['cuda']:
            hist = hist.cuda(self.device)
            nbrs = nbrs.cuda(self.device)
            mask = mask.cuda(self.device)
            lat_enc = lat_enc.cuda(self.device)
            lon_enc = lon_enc.cuda(self.device)
        fut_pred  = self.net(hist, nbrs, mask, lat_enc, lon_enc)
        return fut_pred


print("Execute done")

Execute done


In [19]:
#prediction/traphic.py

# -*- coding: utf-8 -*-
from __future__ import division
import torch
from torch.autograd import Variable
import torch.nn as nn
from tensorboardX import SummaryWriter


class traphicNet(nn.Module):
    def __init__(self, args):
        print("===================Execute Model Iitialization===========")
        super(traphicNet, self).__init__()
        self.args = args
        self.use_cuda = args['cuda']
        self.use_maneuvers = args['use_maneuvers']
        self.train_flag = True

        self.dropout_prob = args['dropout_prob']
        self.encoder_size = args['encoder_size']
        self.decoder_size = args['decoder_size']
        self.in_length = args['in_length']
        self.out_length = args['out_length']
        self.grid_size = args['grid_size']
        self.upp_grid_size = args['upp_grid_size']
        self.soc_conv_depth = args['soc_conv_depth']
        self.conv_3x1_depth = args['conv_3x1_depth']
        self.dyn_embedding_size = args['dyn_embedding_size']
        self.input_embedding_size = args['input_embedding_size']
        self.num_lat_classes = args['num_lat_classes']
        self.num_lon_classes = args['num_lon_classes']
        self.soc_embedding_size = (((args['grid_size'][0]-4)+1)//2)*self.conv_3x1_depth
        self.upp_soc_embedding_size = (((args['upp_grid_size'][0]-4)+1)//2)*self.conv_3x1_depth
        self.ours = args['ours']

        self.ip_emb = torch.nn.Linear(2, self.input_embedding_size)

        if self.ours:
            self.ip_emb_vel = torch.nn.Linear(2, self.input_embedding_size)
            self.ip_emb_nc = torch.nn.Linear(2, self.input_embedding_size)

        self.enc_gru = torch.nn.GRU(self.input_embedding_size, self.encoder_size, 1)

        self.dyn_emb = torch.nn.Linear(self.encoder_size, self.dyn_embedding_size)
        self.bn_conv = torch.nn.BatchNorm2d(self.encoder_size)

        if self.ours:
            self.beh_1 = torch.nn.Linear(self.encoder_size, self.encoder_size)

        self.soc_conv = torch.nn.Conv2d(self.encoder_size, self.soc_conv_depth, 3)
        self.conv_3x1 = torch.nn.Conv2d(self.soc_conv_depth, self.conv_3x1_depth, (3,1))
        self.soc_maxpool = torch.nn.MaxPool2d((2,1), padding=(1,0))

        if self.use_maneuvers:
            self.dec_gru = torch.nn.GRU(self.upp_soc_embedding_size + self.soc_embedding_size + self.dyn_embedding_size + self.num_lat_classes + self.num_lon_classes, self.decoder_size)
        else:
            self.dec_gru = torch.nn.GRU(self.soc_embedding_size + self.dyn_embedding_size, self.decoder_size, dropout=self.dropout_prob)

        self.bnupp_soc_enc = torch.nn.BatchNorm1d(self.input_embedding_size)
        self.bn_soc_enc = torch.nn.BatchNorm1d(self.soc_embedding_size)
        self.bn_hist_enc = torch.nn.BatchNorm1d(self.upp_soc_embedding_size)

        self.op = torch.nn.Linear(self.decoder_size, 5)
        self.bn_lin = torch.nn.BatchNorm1d(self.out_length)

        self.dropout = nn.Dropout(self.dropout_prob)

        if self.ours:
            self.op_lat = torch.nn.Linear(self.upp_soc_embedding_size + self.soc_embedding_size + self.dyn_embedding_size, self.num_lat_classes)
            self.op_lon = torch.nn.Linear(self.upp_soc_embedding_size + self.soc_embedding_size + self.dyn_embedding_size, self.num_lon_classes)
        else:
            self.op_lat = torch.nn.Linear(self.soc_embedding_size + self.dyn_embedding_size, self.num_lat_classes)
            self.op_lon = torch.nn.Linear(self.soc_embedding_size + self.dyn_embedding_size, self.num_lon_classes)

        self.leaky_relu = torch.nn.ELU()
        self.softmax = torch.nn.Softmax(dim=1)

        self.summary = None
        if self.args['tensorboard']:
            self.summary = SummaryWriter()


    def forward(self, hist, upp_nbrs, nbrs, upp_masks, masks, lat_enc, lon_enc):
#         print("===============Execute Forward Pass =============")
        # Forward pass hist
        if self.ours:
            temp = self.leaky_relu(torch.cat((self.ip_emb(hist[0:self.in_length,:,:]), self.ip_emb_vel(hist[self.in_length:,:,:])), 0))
            _, hist_enc = self.enc_gru(temp)  # Add unsqueeze to make the input 3-D
            hist_enc = hist_enc.squeeze(0)  # Remove extra dimension added by unsqueeze
        else:
            _, hist_enc = self.enc_gru(self.leaky_relu(self.ip_emb(hist)))
        
#         print("===============Execute Forward Pass Debug point 1 =============")
        hist_enc = self.leaky_relu(self.dyn_emb(hist_enc.view(hist_enc.shape[0], hist_enc.shape[1])))
#         print("===============Execute Forward Pass Debug point 2 =============")
        # Forward pass nbrs
        if self.ours:
#             print("===============Execute Forward Pass Debug point 1 =============")
            _, nbrs_enc = self.enc_gru(self.leaky_relu(torch.cat((self.ip_emb(nbrs[0:self.in_length,:,:]), self.ip_emb_vel(nbrs[self.in_length:,:,:])), 0)))  # Add unsqueeze to make the input 3-D
#             print("===============Execute Forward Pass Debug point 2 =============")
            nbrs_enc = nbrs_enc.squeeze(0)  # Remove extra dimension added by unsqueeze
        else:
            _, (nbrs_enc, _) = self.enc_gru(self.leaky_relu(self.ip_emb(nbrs)))

        nbrs_enc = nbrs_enc.view(nbrs_enc.shape[0], nbrs_enc.shape[1])
        
        if self.ours:
            a = self.ip_emb(upp_nbrs[0:self.in_length,:,:])
            b = self.ip_emb_vel(upp_nbrs[self.in_length:,:,:])
            c = self.leaky_relu(torch.cat((a,b)))
            _, upp_nbrs_enc = self.enc_gru(c)  # Add unsqueeze to make the input 3-D
            upp_nbrs_enc = upp_nbrs_enc.squeeze(0)  # Remove extra dimension added by unsqueeze
            upp_nbrs_enc = upp_nbrs_enc.view(upp_nbrs_enc.shape[0], upp_nbrs_enc.shape[1])

            nbrs_enc = self.leaky_relu(self.beh_1(nbrs_enc))
            upp_nbrs_enc = self.leaky_relu(self.beh_1(upp_nbrs_enc))
#         print("===============Execute Forward Pass Debug point 20 =============")
        # Masked scatter
        masks2 = masks.bool()
        soc_enc = torch.zeros_like(masks).float()
        soc_enc = soc_enc.masked_scatter_(masks2, nbrs_enc)
        soc_enc = soc_enc.permute(0,3,2,1)

        if self.ours:
            upp_masks2 = upp_masks.bool()
            upp_soc_enc = torch.zeros_like(upp_masks).float()
            upp_soc_enc = upp_soc_enc.masked_scatter_(upp_masks2, upp_nbrs_enc)
            upp_soc_enc = upp_soc_enc.permute(0,3,2,1)

        # Apply convolutional social pooling:maxpool
        soc_enc = self.soc_maxpool(self.leaky_relu(self.dropout(self.conv_3x1(self.bn_conv(self.leaky_relu(self.soc_conv(soc_enc)))))))
        soc_enc = soc_enc.view(-1, self.soc_embedding_size)
#         print("===============Execute Forward Pass Debug point 3 =============")
        if self.ours:
            upp_soc_enc = self.soc_maxpool(self.leaky_relu(self.dropout(self.conv_3x1(self.bn_conv(self.leaky_relu(self.soc_conv(upp_soc_enc)))))))
            upp_soc_enc = upp_soc_enc.view(-1, self.upp_soc_embedding_size)
#         print("===============Execute Forward Pass Debug point 30 =============")/
        # Apply fc soc pooling
        if self.ours:
#             print("===============Execute Forward Pass Debug point 31 =============")
#             print(upp_soc_enc.shape)
#             print(soc_enc.shape)
#             print(hist_enc.shape)
            enc = torch.cat((self.bnupp_soc_enc(upp_soc_enc), self.bn_soc_enc(soc_enc)), 1)
        else:
            enc = torch.cat((soc_enc, hist_enc), 1)

        if self.use_maneuvers:
#             print("===============Execute Forward Pass Debug point 32 =============")
            # Maneuver recognition
            lat_pred = self.softmax(self.op_lat(enc))
            lon_pred = self.softmax(self.op_lon(enc))

#             print("===============Execute Forward Pass Debug point 30 =============")
            if self.train_flag:
                # Concatenate maneuver encoding of the true maneuver
                enc = torch.cat((enc, lat_enc, lon_enc), 1)
                fut_pred = self.decode(enc)
#                 print("Debug fut_pred_print pos-1")
#                 print(f"fut_pred value: {fut_pred}")
                return fut_pred, lat_pred, lon_pred
            else:
                fut_pred = []
                # Predict trajectory distributions for each maneuver class
                for k in range(self.num_lon_classes):
                    for l in range(self.num_lat_classes):
                        lat_enc_tmp = torch.zeros_like(lat_enc)
                        lon_enc_tmp = torch.zeros_like(lon_enc)
                        lat_enc_tmp[:, l] = 1
                        lon_enc_tmp[:, k] = 1
                        enc_tmp = torch.cat((enc, lat_enc_tmp, lon_enc_tmp), 1)
                        fut_pred.append(self.decode(enc_tmp))
#                 print("Debug fut_pred_print pos-2")
#                 print(f"fut_pred value: {fut_pred}")
                return fut_pred, lat_pred, lon_pred
        else:
#             print(f"enc value: {enc}")
            fut_pred = self.decode(enc)
#             print("Debug fut_pred_print pos-3")
#             print(f"fut_pred value: {fut_pred}")
            return fut_pred


    def decode(self, enc):
        enc = enc.repeat(self.out_length, 1, 1)
        h_dec, _ = self.dec_gru(enc)
        h_dec = h_dec.permute(1, 0, 2)
        fut_pred = self.op(h_dec)
        fut_pred = self.bn_lin(fut_pred)
        fut_pred = fut_pred.permute(1, 0, 2)
        fut_pred = self.dropout(fut_pred)
        fut_pred = outputActivation(fut_pred)
        return fut_pred

print("Execute done")

Execute done


In [20]:
#model/import_data.py
# -*- coding: utf-8 -*-
import re
import numpy as np
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os
import sys
# np.set_printoptions(threshold=sys.maxsize)


def import_data(file_dir, homography_dir, out_dir, class_type):
    transform(file_dir, homography_dir, out_dir, class_type)

def merge(file_names, output_dir, dtype, threadid, class_type):
    '''
    file_names: .npy file's lists
    output_dir: txt set list, without .txt
    결국, merge가 하는 일은 dsId.txt들을 하나의 .npy로 정리한 것.
    
    raw dataset:
    dsId1.txt, dsId2.txt, dsId3.txt, ..., dsIdN.txt

    final dataset for NN:
    Set#-traj.npy: 
    [ [ [dsId1, vehId(1), frame1, x, y, ----- i's features -----],
        [dsId1, vehId(2), frame1, x, y, ----- i's features -----],
        [dsId1, vehId(3), frame1, x, y, ----- i's features -----],
                                ... ,
        [dsId1, vehId(f1), frame1, x, y, ----- i's features -----],
        [dsId1, vehId(1), frame2, x, y, ----- i's features -----],
        [dsId1, vehId(2), frame2, x, y, ----- i's features -----],
                                ... ,
        [dsId1, vehId(fM_1), frameM_1, x, y, ----- i's features -----],
        

        [dsId2, vehId(1), frame1, x, y, ----- i's features -----],
                                ... ,
        [dsId2, vehId(fM_2), frameM_2, x, y, ----- i's features -----],


                                ... ,
        [dsIdN, vehId(i), frame, x, y, ----- i's features -----] ] ]

    Set#-track.npy:
      [ {
            dsId1: {
                    vehId(1): [frame, x, y]'s history,
                    vehId(2): [frame, x, y]'s history,
                                    ... ,
                    vehId(p1): [frame, x, y]'s history
                    }

            dsId2:  {
                    vehId(1): [frame, x, y]'s history,
                    vehId(2): [frame, x, y]'s history,
                                    ... ,
                    vehId(p2): [frame, x, y]'s history
                    }
                ...
            
            dsIdN:  {
                    vehId(1): [frame, x, y]'s history,
                    vehId(2): [frame, x, y]'s history,
                                    ... ,
                    vehId(pN): [frame, x, y]'s history
                    }
        } ]
    '''
    output_dir = output_dir + '/{}'
    traj = np.array([])

    track = defaultdict(dict)

    i = 0
    sz = len(file_names)
    for f in file_names:#get .npy file
        print("Start merging {}/{} in {} in thread {}...".format(i, sz, dtype, threadid))
        i += 1
        # print("Reading dataset {}...".format(d))
        npy_path = f
        print('reading... ', npy_path)
        data = np.load(npy_path, allow_pickle=True)# get formated .npy file

        # print('data', data.shape)
        # constructing train, val and testset for trajectory
        data0 = data[0]#traj data for a specific class
        data2 = data[2]#traj data for all
        
        traj_id = np.unique(data2[:,1])# get all object_id from dset 'd' if you make dataset for specific agent type
        
        if len(data0)==0:
            continue
        d = int(data0[0, 0])#dataset id
        

        if traj.size == 0:
            traj = data0
        else:
            '''# traj는 모든 #.npy의 traj를 합친 파일.'''
            traj = np.concatenate((traj, data0), axis=0)

        # constructing train, val and testset for tracks
        data1 = data[1]#track dataset from 'transform' function
        for ids in traj_id:#(t, x, y)
            '''# track은 모든 .npy의 track을 저장한 파일.'''
            track[d][ids] = data1[ids]#literally, each object's trajectory

        # print("Dataset {} finsihed.".format(d))
    
    if not os.path.exists(output_dir.format(dtype)):
        os.makedirs(output_dir.format(dtype))

    # data for sgan
    # sgan_name = "{}/{}Set{}.txt".format(dtype, dtype, str(threadid))
    # f = open(output_dir.format(sgan_name), 'w')
    # for line in traj:
    #     # f.write("{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4]))
    #     f.write("{}\t{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4], int(line[0])))
    # f.close()

    if class_type == 'vehicle':
        npy_name = "{}/{}Set{}-traj-v.npy".format(dtype, dtype, str(threadid))
    elif class_type == 'bike/motor':
        npy_name = "{}/{}Set{}-traj-b.npy".format(dtype, dtype, str(threadid))
    elif class_type == 'human':
        npy_name = "{}/{}Set{}-traj-h.npy".format(dtype, dtype, str(threadid))
    else:
        npy_name = "{}/{}Set{}-traj.npy".format(dtype, dtype, str(threadid))

    name = npy_name
    np.save(output_dir.format(name), np.array([traj]))
    name = "{}/{}Set{}-track.npy".format(dtype, dtype, str(threadid))
    np.save(output_dir.format(name), np.array([track]))
    
    print("{} file in thread {} is saved and ready.".format(dtype, threadid))

    return len(traj)

def merge_n_split(file_names, output_dir):

    output_dir = output_dir + '/{}'
    traj_train = np.array([])
    traj_val = np.array([])
    traj_test = np.array([])

    track_train = defaultdict(dict)
    track_val = defaultdict(dict)
    track_test = defaultdict(dict)

    print("Start spliting data...")

    for f in file_names:
        # print("Reading dataset {}...".format(d))
        npy_path = f
        # print(npy_path)
        data = np.load(npy_path, allow_pickle=True)

        # constructing train, val and testset for trajectory
        traj = data[0]
        traj_id = np.unique(traj[:,1])
        d = int(traj[0,0])

        # split the dataset using vehicle id
        traj_id, test_id = train_test_split(traj_id, test_size=0.2, random_state=0)
        train_id, val_id = train_test_split(traj_id, test_size=0.125, random_state=0)

        # get trajectory with vehicle id
        train = np.array(traj[ [(s in train_id) for s in traj[:,1]] ])
        if traj_train.size == 0:
            traj_train = train
        else:
            traj_train = np.concatenate((traj_train, train), axis=0)

        val = np.array(traj[ [(s in val_id) for s in traj[:,1]] ])
        if traj_val.size == 0:
            traj_val = val
        else:
            traj_val = np.concatenate((traj_val, val), axis=0)

        test = np.array(traj[ [(s in test_id) for s in traj[:,1]] ])
        if traj_test.size == 0:
            traj_test = test
        else:
            traj_test = np.concatenate((traj_test, test), axis=0)


        # constructing train, val and testset for tracks
        track = data[1]
        for i in train_id:
            track_train[d][i] = track[i]

        for i in val_id:
            track_val[d][i] = track[i]

        for i in test_id:
            track_test[d][i] = track[i]     

        print("Dataset {} finsihed.".format(d))



    if not os.path.exists(output_dir.format("train")):
        os.makedirs(output_dir.format("train"))

    f = open(output_dir.format("train/TrainSet.txt"), 'w')
    for line in traj_train:
        # f.write("{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4]))
        f.write("{}\t{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4], int(line[0])))
    f.close()

    if not os.path.exists(output_dir.format("val")):
        os.makedirs(output_dir.format("val"))

    f = open(output_dir.format("val/ValSet.txt"), 'w')
    for line in traj_val:
        # f.write("{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4]))
        f.write("{}\t{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4], int(line[0])))
    f.close()

    if not os.path.exists(output_dir.format("test")):
        os.makedirs(output_dir.format("test"))    

    f = open(output_dir.format("test/TestSet.txt"), 'w')
    for line in traj_test:
        # f.write("{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4]))
        f.write("{}\t{}\t{}\t{}\t{}\n".format(int(line[2]), int(line[1]), line[3], line[4], int(line[0])))
    f.close()

    np.save(output_dir.format("TrainSet.npy"), np.array([traj_train, track_train]))
    np.save(output_dir.format("ValSet.npy"), np.array([traj_val, track_val])) 
    np.save(output_dir.format("TestSet.npy"), np.array([traj_test, track_test]))
    print("Training file saved and ready.")

    # print(traj_train)
    # print(traj_val)
    # print(traj_test)
    # print(len(traj_train))
    # print(len(traj_val))
    # print(len(traj_test))
    # for d in 
    # print(len(traj_train))
    # print(len(traj_val))
    # print(len(traj_test))
    # print(traj_val)


def filter_edge_cases(traj, track): 
    size = np.shape(traj)[0]
    idx = np.zeros((size, 1))

    for k in range(size):
        t = traj[k,2]

        if np.shape(track[traj[k,1]])[1] > 30 and track[traj[k,1]][0, 30] < t and track[traj[k,1]][0,-1] > t+1:
            idx[k] = 1       

    return traj[np.where(idx == 1)[0],:]




def px_to_ft(traj, homography_dir):

    m_to_ft = 3.28084
    h = np.loadtxt(homography_dir, delimiter=' ')
    c_x = 1280/2
    c_y = 720/2

    traj[:,3] = traj[:,3] - c_x
    traj[:,4] = traj[:,4] - c_y
    traj[:,3:5] = multiply_homography(h, traj[:,3:5]) * m_to_ft
    print("Finish converting pixel to feet")
    return traj

def multiply_homography(h, pt_in):
    # a = np.transpose(pt_in)
    # b = np.ones((1, np.shape(pt_in)[0]))
    # print(np.concatenate((a,b)))
    pt = np.matmul(h, np.concatenate((np.transpose(pt_in), np.ones((1, np.shape(pt_in)[0])))))
    pt = np.transpose(pt[0:2,:])
    # print(pt)
    return pt

def transform(file_dir, homography_dir, out_dir, class_type):
    #transform(formated_txt_file_dir, None, formated_npy file_dir)
    read = np.loadtxt(file_dir, delimiter=',')
    traj = np.zeros((np.shape(read)[0], 47))
    traj[:,:5] = read[:,:5]
    
    # uniq_id = np.unique(traj)
    ############ preprocess ##################
    read[read[:,5]==2,5]=1# only vehicle
    read_v = read[read[:,5]==1]
    traj_v = np.zeros((np.shape(read_v)[0], 47))
    traj_v[:,:5] = read_v[:,:5]

    read_b = read[read[:,5]==4]# only bike/motorcycle
    traj_b = np.zeros((np.shape(read_b)[0], 47))
    traj_b[:,:5] = read_b[:,:5]
    
    read_h = read[read[:,5]==3]# only human
    traj_h = np.zeros((np.shape(read_h)[0], 47))
    traj_h[:,:5] = read_h[:,:5]
    
    traj_class = None
    if class_type == 'vehicle':
        traj_class = traj_v
    elif class_type == 'bike/motor':
        traj_class = traj_b
    elif class_type == 'human':
        traj_class = traj_h
    else:# for all
        traj_class = traj
    

    # for k in range(np.shape(traj)[0]):#Ben: each row index(k)
    for k in range(np.shape(traj_class)[0]):#Ben: each row index(k)
        # print("Progress: {}/{} ...".format((k+1), np.shape(traj)[0]))
        dsid = traj_class[k][0]#traj_v
        vehid = traj_class[k][1]#traj_v
        time = traj_class[k][2]#traj_v
        
        #Get observed vehicle's trajectory
        vehtraj = traj_class[traj_class[:,1] == vehid]
        
        #Get all rows which are in the same frame
        frameEgo = traj[traj[:,2] == time]
        
        ''' Get Features '''
        if frameEgo.size != 0:#In one frame, there are a lot of dynamic obstacles
            dx = np.zeros(np.shape(frameEgo)[0])
            dy = np.zeros(np.shape(frameEgo)[0])
            vid = np.zeros(np.shape(frameEgo)[0])
            
            # agent_i를 기준으로 같은 frame 내의 object들과의 dx, dy 구하기.
            for l in range(np.shape(frameEgo)[0]):
                dx[l] = frameEgo[l][3] - traj_class[k][3]#traj_v
                dy[l] = frameEgo[l][4] - traj_class[k][4]
                vid[l] = frameEgo[l][1]
            dist = dx*dx + dy*dy# Get the distance between others from that vehicle;
            
            lim = 39# maximum 39 dynamic obstacles only

            if len(dist) > lim:
                idx = np.argsort(dist)
                # print(idx)
                dx = np.array([dx[i] for i in idx[:lim]])
                dy = np.array([dy[i] for i in idx[:lim]])
                vid = np.array([vid[i] for i in idx[:lim]])

            # left
            xl = dx[dx < 0]
            yl = dy[dx < 0]
            vidl = vid[dx < 0]

            yl_top = yl[yl>=0]
            yl_bot = yl[yl<0]
            vidl_top = vidl[yl>=0]#index
            vidl_bot = vidl[yl<0]

            # center
            xc = dx[dx >= 0]
            yc = dy[dx >= 0]
            vidc = vid[dx >= 0]
            yc = yc[xc < 200]
            vidc = vidc[xc < 200]
            xc = xc[xc < 200]

            yc_top = yc[yc>=0]
            yc_bot = yc[yc<0]
            vidc_top = vidc[yc>=0]
            vidc_bot = vidc[yc<0]

            # right
            xr = dx[dx >= 200]
            yr = dy[dx >= 200]
            vidr = vid[dx >= 200]

            yr_top = yr[yr>=0]
            yr_bot = yr[yr<0]
            vidr_top = vidr[yr>=0]
            vidr_bot = vidr[yr<0]


            # parameters
            mini_top = 7
            mini_bot = 6

            # left top
            iy = np.argsort(yl_top)
            iy = iy[0:min(mini_top, len(yl_top))]# 최대 6개의 좌측에 존재하는 장애물을 고려.
            yl_top = np.array([yl_top[i] for i in iy])
            vidl_top = np.array([vidl_top[i] for i in iy])
            # left bottom
            iy = np.argsort(yl_bot)
            iy = np.array(list(reversed(iy)))
            iy = iy[0:min(mini_bot, len(yl_bot))]
            yl_bot = np.array([yl_bot[i] for i in iy])
            vidl_bot = np.array([vidl_bot[i] for i in iy])

            # center top
            iy = np.argsort(yc_top)
            iy = iy[0:min(mini_top, len(yc_top))]
            yc_top = np.array([yc_top[i] for i in iy])
            vidc_top = np.array([vidc_top[i] for i in iy])
            # center bottom
            iy = np.argsort(yc_bot)
            iy = np.array(list(reversed(iy)))
            iy = iy[0:min(mini_bot, len(yc_bot))]
            yc_bot = np.array([yc_bot[i] for i in iy])
            vidc_bot = np.array([vidc_bot[i] for i in iy])

            # right top
            iy = np.argsort(yr_top)
            iy = iy[0:min(mini_top, len(yr_top))]
            yr_top = np.array([yr_top[i] for i in iy])
            vidr_top = np.array([vidr_top[i] for i in iy])
            # right bottom
            iy = np.argsort(yr_bot)
            iy = np.array(list(reversed(iy)))
            iy = iy[0:min(mini_bot, len(yr_bot))]
            yr_bot = np.array([yr_bot[i] for i in iy])
            vidr_bot = np.array([vidr_bot[i] for i in iy])


            #traj_v
            traj_class[k,8:14] = np.concatenate((np.zeros(6 - len(vidl_bot)),vidl_bot))#object_i의 left bottom에 위치한 다른 object들의 id 정보.
            traj_class[k,14:21] = np.concatenate((vidl_top ,np.zeros(7 - len(vidl_top))))#left top
            traj_class[k,21:27] = np.concatenate((np.zeros(6 - len(vidc_bot)),vidc_bot))#center bottom
            traj_class[k,27:34] = np.concatenate((vidc_top ,np.zeros(7 - len(vidc_top))))#center top
            traj_class[k,34:40] = np.concatenate((np.zeros(6 - len(vidr_bot)),vidr_bot))#right bottom
            traj_class[k,40:47] =np.concatenate((vidr_top ,np.zeros(7 - len(vidr_top))))#right top

    # convert from pixel to feet

    if homography_dir:#None
        traj = px_to_ft(traj, homography_dir)


    # create track
    ids = np.unique(traj[:,1])# Get all vehicle's id
    track = {} 
    for i in range(len(ids)):
        vtrack = traj[traj[:,1] == ids[i]]# get all row for 'ids[i]'
        track[ids[i]] = vtrack[:,2:5].T # get (t, x, y)


    # np.save(out_dir, np.array([traj, track]))
    np.save(out_dir, np.array([traj_class, track, traj]))

    # transfrom의 output
    '''
    traj  =  dsId.txt 파일과 같은 크기의 row. 각 row마다 [ dsId, object id(i), frame, x, y, i의 주변 39개까지의 주변 장애물과의 위치 관계]
    track =  dsId.txt 파일에 존재하는 obejct(i)의 trajectory {object_id(i): [ [frame_1, x_1, y_1], [frame_2, x_2, y_2], ... [frame_p, x_p, y_p]  ]}
    '''

print("Execute done")

Execute done


In [21]:
#model/model.py
import re
import os
import subprocess
import torch
import argparse


# from model.Prediction.traphic import traphicNet
# from model.Prediction.social import highwayNet
# from model.Prediction.utils import ngsimDataset
# from model.Prediction.traphicEngine import TraphicEngine
# from model.Prediction.socialEngine import SocialEngine
from torch.utils.data import DataLoader
import datetime




class TnpModel:

    def __init__(self, inArgs):
        torch.manual_seed(inArgs['dsId'])#seed becomes 'dataset Id'
        torch.cuda.manual_seed(inArgs['dsId'])

        self.args = {}
        self.args["batch_size"] = inArgs["batch_size"]
        self.args["pretrainEpochs"] = inArgs["pretrainEpochs"]
        self.args["trainEpochs"] = inArgs["trainEpochs"]
        self.args['cuda'] = inArgs["cuda"]
        self.args['device'] = inArgs['device']
        self.args['modelLoc'] = inArgs['modelLoc']#Ben: the location of 'trained model'
        self.args["optim"] = inArgs["optim"]

        # Network Arguments
        self.args['dropout_prob'] = inArgs["dropout"]
        self.args['encoder_size'] = 64
        self.args['decoder_size'] = 128
        self.args['in_length'] = inArgs['input_size']#Ben: INPUT (history length)
        self.args['out_length'] = inArgs['output_size']#Ben: OUTPUT (output seq length)
        self.args['grid_size'] = (13,3)
        self.args['upp_grid_size'] = (7,3)
        self.args['soc_conv_depth'] = 64
        self.args['conv_3x1_depth'] = 16
        self.args['dyn_embedding_size'] = 32
        self.args['input_embedding_size'] = 32
        self.args['num_lat_classes'] = 3
        self.args['num_lon_classes'] = 2
        self.args['use_maneuvers'] = inArgs["maneuvers"]
        self.args['ours'] = (inArgs["predAlgo"] == "Traphic")
        self.args['nll_only'] = inArgs['nll_only']
        self.args["learning_rate"] = inArgs["lr"]
        self.args["predAlgo"] = inArgs["predAlgo"]#TraPHic
        self.args["w_decay"] = inArgs['w_decay']
        
        # currentDT = datetime.datetime.now()
        # self.args['name'] = "{}_{}_model.tar".format(inArgs["predAlgo"], currentDT.strftime("%Y_%m_%d_%H_%M"))
        self.args['name'] = inArgs['name_temp'].format(self.args["predAlgo"], inArgs['dset'])
        self.args["pretrain_loss"] = inArgs['pretrain_loss']
        self.args['train_loss'] = inArgs['train_loss']
        self.args['dir'] = inArgs['dir']
        self.args['raw_dir'] = inArgs['raw_dir']
        self.args['dsId'] = inArgs['dsId']
        self.args['log_dir'] = inArgs['log_dir']
        self.args['tensorboard'] = inArgs['tensorboard']
        self.args['class_type'] = inArgs['class_type']
        if self.args["predAlgo"] == "Traphic":# Ben: Declare the network
            self.net = traphicNet(self.args)
        else:
            self.net = highwayNet(self.args)

        if self.args['cuda']:
            self.net = self.net.cuda(self.args['device'])
        


    def eval_one(self, dsId=None):
        if dsId:
            self.args['dsId'] = dsId

        self.net.train_flag = False

        self.net.eval()
        d = os.path.join(self.args['modelLoc'], self.args['name'])

        if os.path.exists(d):
            self.net.load_state_dict(torch.load(d))
            print("\n[INFO]: model {} loaded".format(d))
        else:
            print("\n[INFO]: can not find model at {} to evaluate, using existing net".format(d))

        if self.args["cuda"]:
            self.net.cuda(self.args['device'])



        if self.args["optim"] == "Adam":
            optim = torch.optim.Adam(self.net.parameters(),lr=self.args['learning_rate'],weight_decay=self.args["w_decay"])
        elif self.args["optim"] == "SGD":
            optim = torch.optim.SGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "AdamW":
            optim = torch.optim.AdamW(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "SparseAdam":
            optim = torch.optim.SparseAdam(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Adamax":
            optim = torch.optim.Adamax(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "ASGD":
            optim = torch.optim.ASGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Rprop":
            optim = torch.optim.Rprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "RMSprop":
            optim = torch.optim.RMSprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "LBFGS":
            optim = torch.optim.LBFGS(self.net.parameters(),lr=self.args['learning_rate'])
        else:
            print("undefined optimizer.")
            return

        crossEnt = torch.nn.BCELoss()
        self.net()

    def load(self, d=None, load=False):
        self.net.eval()#Ben: model.eval() will notify all your layers that you are in eval mode, 
                       #     that way, batchnorm or dropout layers will work in eval mode instead of training mode.
        if not d:#Ben: Get a location(path)
            d = os.path.join(self.args['modelLoc'], self.args['name'])
        else:
            if load:
                self.args['name'] = d 
            d = os.path.join(self.args['modelLoc'], d)
        
        if os.path.exists(d):
            self.net.load_state_dict(torch.load(d))
            print("\n[INFO]: model {} loaded\n".format(d))
        else:
            print("\n[INFO]: can not find model at {} to evaluate, using existing net".format(d))


    def train(self, dsId=None):
        if dsId:
            self.args['dsId'] = dsId
    
        self.net.train_flag = True
        self.net.train()
        if self.args["cuda"]:
            self.net.cuda(self.args['device'])

        if self.args["optim"] == "Adam":
            optim = torch.optim.Adam(self.net.parameters(),lr=self.args['learning_rate'],weight_decay=self.args["w_decay"])
        elif self.args["optim"] == "SGD":
            optim = torch.optim.SGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "AdamW":
            optim = torch.optim.AdamW(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "SparseAdam":
            optim = torch.optim.SparseAdam(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Adamax":
            optim = torch.optim.Adamax(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "ASGD":
            optim = torch.optim.ASGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Rprop":
            optim = torch.optim.Rprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "RMSprop":
            optim = torch.optim.RMSprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "LBFGS":
            optim = torch.optim.LBFGS(self.net.parameters(),lr=self.args['learning_rate'])
        else:
            print("undefined optimizer.")
            return

        crossEnt = torch.nn.BCELoss()#ben: Binary Cross Entrophy

        print('loading data in {}...'.format(self.args['dsId']))
        trSet_path = os.path.join(self.args["dir"], "trainSet")
        valSet_path = os.path.join(self.args["dir"], "valSet")
        
        trSet = ngsimDataset(trSet_path, self.args["dir"], self.args["raw_dir"], 'train', self.args['dsId'], self.args['class_type'], t_h=self.args['in_length'], t_f=self.args['out_length'])
        valSet = ngsimDataset(valSet_path, self.args["dir"], self.args["raw_dir"], 'val', self.args['dsId'], self.args['class_type'], t_h=self.args['in_length'], t_f=self.args['out_length'])

        trDataloader = DataLoader(trSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=trSet.collate_fn)
        # trDataloader = DataLoader(valSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=valSet.collate_fn)
        valDataloader = DataLoader(valSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=valSet.collate_fn)
        
        print("==================Attribute===========")
        sample = trSet[0]  # Get the first sample
        num_attributes = len(sample)  # Get the number of attributes in the sample

        print("Number of Attribute")
        print(num_attributes)
        print("==================Attribute===========")
        
        
        print('start training {}...'.format(self.args["predAlgo"]))
        if self.args["predAlgo"] == "Traphic":
            engine = TraphicEngine(self.net, optim, trDataloader, valDataloader, self.args)
        else:
            engine = SocialEngine(self.net, optim, trDataloader, valDataloader, self.args)

        engine.start()


        

    def evaluate(self, dsId=None):
        if dsId:# Ben: dataset id?? TODO
            self.args['dsId'] = dsId

        self.net.train_flag = False
        self.net.eval()# Ben: Ready for evaluation
        d = os.path.join(self.args['modelLoc'], self.args['name'])

        if os.path.exists(d):
            self.net.load_state_dict(torch.load(d, map_location = 'cuda:0'))#Ben: error handled
            print("\n[INFO]: model {} loaded".format(d))
        else:
            print("\n[INFO]: can not find model at {} to evaluate, using existing net".format(d))

        if self.args["cuda"]:
            self.net.cuda(self.args['device'])



        if self.args["optim"] == "Adam":
            optim = torch.optim.Adam(self.net.parameters(),lr=self.args['learning_rate'],weight_decay=self.args["w_decay"])
        elif self.args["optim"] == "SGD":
            optim = torch.optim.SGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "AdamW":
            optim = torch.optim.AdamW(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "SparseAdam":
            optim = torch.optim.SparseAdam(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Adamax":
            optim = torch.optim.Adamax(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "ASGD":
            optim = torch.optim.ASGD(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "Rprop":
            optim = torch.optim.Rprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "RMSprop":
            optim = torch.optim.RMSprop(self.net.parameters(),lr=self.args['learning_rate'])
        elif self.args["optim"] == "LBFGS":
            optim = torch.optim.LBFGS(self.net.parameters(),lr=self.args['learning_rate'])
        else:
            print("undefined optimizer.")
            return

        crossEnt = torch.nn.BCELoss()

        print('loading data in {}...'.format(self.args['dsId']))
        trSet_path = os.path.join(self.args["dir"], "trainSet")
        valSet_path = os.path.join(self.args["dir"], "valSet")
        tstSet_path = os.path.join(self.args["dir"], "testSet")

        trSet = ngsimDataset(trSet_path, self.args["dir"], self.args["raw_dir"], 'train', self.args['dsId'], self.args['class_type'], t_h=self.args['in_length'], t_f=self.args['out_length'])
        trDataloader = DataLoader(trSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=trSet.collate_fn)

        testSet = ngsimDataset(valSet_path, self.args["dir"], self.args["raw_dir"], 'val', self.args['dsId'], self.args['class_type'], t_h=self.args['in_length'], t_f=self.args['out_length'])
        testDataloader = DataLoader(testSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=testSet.collate_fn)

        valSet = ngsimDataset(tstSet_path, self.args["dir"], self.args["raw_dir"], 'val', self.args['dsId'], self.args['class_type'], t_h=self.args['in_length'], t_f=self.args['out_length'])
        valDataloader = DataLoader(valSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=valSet.collate_fn)

        print('start testing {}...'.format(self.args["predAlgo"]))
        if self.args["predAlgo"] == "Traphic":
            engine = TraphicEngine(self.net, optim, trDataloader, valDataloader, self.args)
        else:
            engine = SocialEngine(self.net, optim, trDataloader, valDataloader, self.args)

        engine.eval(trDataloader)
        # engine.eval(testDataloader)

    def result_viz(self):#TODO
        # This function is for visualizing the network output and ground truth trajectory
        # print('loading data in {}...'.format(self.args['dsId']))
        tstSet_path = os.path.join(self.args["dir"], "testSet")

        testSet = ngsimDataset(valSet_path, self.args["dir"], self.args["raw_dir"], 'val', self.args['dsId'], t_h=self.args['in_length'], t_f=self.args['out_length'])
        testDataloader = DataLoader(testSet,batch_size=self.args['batch_size'],shuffle=True,num_workers=4,collate_fn=testSet.collate_fn)
        if self.args["predAlgo"] == "Traphic":
            engine = TraphicEngine(self.net, optim, trDataloader, valDataloader, self.args)
        else:
            engine = SocialEngine(self.net, optim, trDataloader, valDataloader, self.args)
        

print("Execute done")

Execute done


In [22]:
#traphic_main.py
import argparse
import warnings
import re
import os
import csv
# from model.model import TnpModel
# from model.import_data import *
import time

warnings.filterwarnings("ignore")

'''     IMPORTANT     '''

DATASET = 'APOL'# Use Apolloscape dataset
LOG = '/kaggle/working/logs/'
LOAD = ''# Load the trained model

CUDA = True 
DEVICE = 'cuda:0'

PREDALGO = 'Traphic'
# PREDALGO = 'Highway'

PRETRAINEPOCHS= 0# Untill pretrained epoch
TRAINEPOCHS= 10# After pretrained epoch
INPUT = 6 #Trajectory sequence input
OUTPUT = 10#Trajectory sequence output for prediction
MANUAL_SEED = 42
TENSORBOARD = True#For using tensorboard

DATA_DIR = '/kaggle/input/apoldata/data/' + DATASET
MODELLOC = "/kaggle/working/TRAPHIC_weight/"
RAW_DATA = "/kaggle/input/apoldata/data/prediction_train/"

TRAIN = True
EVAL = True

# training option 				========== Hyper-parameter ==========
BATCH_SIZE = 32
DROPOUT = 0.5
OPTIM= 'Adam'
# SGD Adam AdamW SparseAdam Adamax ASGD RMSprop Rprop 
LEARNING_RATE= 0.001
MANEUVERS = False#Ben: TODO
PRETRAIN_LOSS = 'NLL'# Negative Log-Likelihood
TRAIN_LOSS = 'MSE'
NLL_ONLY = True
WEIGHT_DECAY = 1e-4
# Trained model name for saving
NAME = '{}.{}' + '.model_{}-{}l_{}epochs.seed{}.batch{}.nll_only.{}.tar'\
			.format(INPUT, OUTPUT, PRETRAINEPOCHS + TRAINEPOCHS, MANUAL_SEED, BATCH_SIZE, NLL_ONLY)

GENERATE_DATASET = False
# If you want to generate a model for considering vehicle' only, "CLASS_TYPE = 'vehicle'",
# For any other class, there are {'bike/motor', 'human'}. For considering all class, just use 'all'
CLASS_TYPE = 'all' #(vehicle, 'bike/motor', 'human', 'all')

def apol_to_formatted(input_dir, files, output_dir, dtype):
	txtlst = []
	i = 0 
	sz = len(files)
	print("=======================================")
	for f in files:
		print("Processing {}/{} in {}...".format(i, sz, dtype))
		# print("files: ", f)
		i += 1
		splitted_name = f.split('_')
		dset_id = splitted_name[1] + splitted_name[2].zfill(2)#for prediction_train,test.zip
		
		out_name = dset_id + '.txt'
		txtlst.append(dset_id)
		
		current_time = -1
		current_frame_num = -1

		if not os.path.exists(output_dir):
			os.mkdir(output_dir)

		out = open(os.path.join(output_dir, out_name), 'w')
		f = os.path.join(input_dir, f)

		with open(f) as csv_file:
			for row in csv.reader(csv_file):
	
				each_row = row[0].split(' ')
				current_frame_num = each_row[0]
				vid_type = each_row[2]
            
				vid = int(each_row[1].split('-')[-1])
				out.write("{},{},{},{},{},{}\n".format(float(dset_id), vid, current_frame_num, each_row[3], each_row[4], vid_type))
	return txtlst

def create_data(input_dir, file_names, output_dir, dtype, threadid, class_type):
	name_lst = []
	i = 0
	sz = len(file_names)
	for f in file_names:
		print("Importing data {}/{} for {} in thread {}...".format(i, sz, dtype, threadid))
		i += 1
		dset_id = f
		
		loc = os.path.join(input_dir,dset_id+'.txt')#from 'formated folder'; i.e. formated txt file
		out = os.path.join(input_dir,dset_id+'.npy')
		import_data(loc, None, out, class_type)
		name_lst.append(out)
	
	merge(name_lst, output_dir, dtype, threadid, class_type)
	print('"merge" is finished!')



if __name__ == "__main__":

	parser = argparse.ArgumentParser(description="traphicPred command line control")

	parser.add_argument('--cuda', '-g', action='store_true', help='GPU option', default=CUDA)
	parser.add_argument('--device', '-d', help='cuda device option', default=DEVICE, type=str)

	parser.add_argument('--batch_size', '-b', help='bastch size', default=BATCH_SIZE)
	parser.add_argument('--dropout', help='dropout probability', default=DROPOUT)
	parser.add_argument('--lr', help='learning rate', default=LEARNING_RATE)
	parser.add_argument('--optim', help='optimiser', default=OPTIM)
	parser.add_argument('--w_decay', help='weight decay rate', default=WEIGHT_DECAY)
	parser.add_argument('--pretrainEpochs', '-p', help='number of epochs for pretraining', default=PRETRAINEPOCHS, type=int)
	parser.add_argument('--trainEpochs', '-e', help='number of epochs for training', default=TRAINEPOCHS, type=int)
	parser.add_argument('--maneuvers', help='maneuvers option', default=MANEUVERS, type=bool)
	parser.add_argument('--predalgo', help='prediction algorithm', default=PREDALGO)#TraPHic
	parser.add_argument('--pretrain_loss', help='pretrain loss algorithm', default=PRETRAIN_LOSS)
	parser.add_argument('--train_loss', help='train loss algorithm', default=TRAIN_LOSS)

	parser.add_argument('--dset', '-s', help='cuda device option', default=DATASET, type=str)
	parser.add_argument('--modelLoc', help='trained prediction store/load location', default=MODELLOC)
	parser.add_argument('--dir', help="location of the dataset for tracking", default=DATA_DIR)
	parser.add_argument("-f", required=False)

	args, unknown = parser.parse_known_args()



	viewArgs = {}
	viewArgs['cuda'] = args.cuda
	viewArgs['log_dir'] = LOG
	viewArgs['batch_size'] = args.batch_size
	viewArgs['dropout'] = args.dropout
	viewArgs["lr"] = args.lr
	viewArgs["optim"] = args.optim
	viewArgs['w_decay'] = args.w_decay
	viewArgs['pretrainEpochs'] = args.pretrainEpochs
	viewArgs['trainEpochs'] = args.trainEpochs
	viewArgs["maneuvers"] = args.maneuvers
	viewArgs['predAlgo'] = args.predalgo
	viewArgs['pretrain_loss'] = args.pretrain_loss
	viewArgs['train_loss'] = args.train_loss
	viewArgs['nll_only'] = NLL_ONLY
	viewArgs['tensorboard'] = TENSORBOARD

	viewArgs['modelLoc'] = args.modelLoc
	viewArgs['dir'] = args.dir
	viewArgs['raw_dir'] = RAW_DATA
	if not args.cuda:
		args.device = 'cpu'
	viewArgs['device'] = args.device

	viewArgs['dsId'] = MANUAL_SEED# It represents seed number, which means each different seed generates different (train, val, test) set
	viewArgs['dset'] = args.dset
	viewArgs['name_temp'] = NAME
	viewArgs['input_size'] = INPUT
	viewArgs['output_size'] = OUTPUT
	viewArgs['class_type'] = CLASS_TYPE
	
	if GENERATE_DATASET:
		''' dataset ratio --> (train, val, test)==(0.7, 0.2, 0.1) '''
		np.random.seed(MANUAL_SEED)
		threadid = MANUAL_SEED
		class_type = CLASS_TYPE
		files = None

		raw_data = os.listdir(RAW_DATA)
		get_all_txtfile = [f for f in raw_data if '.txt' in f]
		dataset_cnt = len(get_all_txtfile)# Ben: Get the number of all data in 'data_dirs'
		datasets_dir = sorted(get_all_txtfile)
		np.random.shuffle(datasets_dir)
	
		datasets_for_train = datasets_dir[:int(dataset_cnt * 0.7)]
		datasets_for_val = datasets_dir[int(dataset_cnt * 0.7):int(dataset_cnt * 0.9)]
		datasets_for_test = datasets_dir[int(dataset_cnt * 0.9) :]
		
		print('dataset is generated...')
		train_loc = RAW_DATA
		output_dir = RAW_DATA + '/train/formatted/'
		files = datasets_for_train
		train_lst = apol_to_formatted(train_loc, files, output_dir, "train")
		create_data(output_dir, train_lst, args.dir, "train", threadid, class_type)

		val_loc = RAW_DATA
		output_dir = RAW_DATA + '/val/formatted/'
		files = datasets_for_val
		val_lst = apol_to_formatted(val_loc, files, output_dir, "val")
		create_data(output_dir, val_lst, args.dir, "val", threadid, class_type)

		test_loc = RAW_DATA
		output_dir = RAW_DATA + '/test_obs/formatted/'
		files = datasets_for_test
		test_lst = apol_to_formatted(test_loc, files, output_dir, "test")
		create_data(output_dir, test_lst, args.dir, "test", threadid, class_type)

		quit()
	print('using {} dataset.'.format(DATASET))

	t0 = time.time()#ben: initialize time

	model = TnpModel(viewArgs)
	if args.cuda:
		print("using cuda...\n")
	else:
		print("using cpu...\n")

	if LOAD != '':
		model.load(LOAD)
		print("===================Load Model===========")
        
	t1 = time.time()

	if TRAIN:
		model.train(viewArgs['dsId'])

	t2 = time.time()

	if EVAL:
		model.evaluate()

	t3 = time.time()

	print('\nusing {} dataset.'.format(DATASET))
	
	print('Loading time:{}'.format(t1 - t0))
	print("Training time:{}".format(t2 - t1))
	print("Testing time:{}".format(t3 - t2))


using APOL dataset.
===================Execute Model Iitialization===========
using cuda...

loading data in 42...
load train dataset
dtype is train and dset(manual_seed) is 42
The size of TRAJ file:  49232
The size of TRACK file:  37
load val dataset
dtype is val and dset(manual_seed) is 42
The size of TRAJ file:  13658
The size of TRACK file:  10
==================Attribute===========
Number of Attribute
10
==================Attribute===========
start training Traphic...


[1/1539]   0%|          , Avg train loss=4.65e+3, Avg val loss=0 [00:00<?]

KeyboardInterrupt: 